In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
 ! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
 ! kaggle datasets list

In [ ]:
! kaggle competitions download -c 'kaggle datasets download -d hamzaboulahia/hardfakevsrealfaces'

In [ ]:
! mkdir train

In [ ]:
! unzip archive.zip -d train

# New Section

In [ ]:
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from tensorflow.keras.utils import to_categorical
import os
import numpy as np



In [ ]:
dataset_directory = '/content/train'

# Load images and labels
all_images = []
all_labels = []

In [ ]:
for label in ['real', 'fake']:
    label_directory = os.path.join(dataset_directory, label)

    # Get a list of all image filenames in the subdirectory
    image_files = os.listdir(label_directory)

    for image_file in image_files:
        # Load image from subdirectory
        img_path = os.path.join(label_directory, image_file)
        img = Image.open(img_path).resize([128, 128])
        img = np.array(img).astype(np.float32)
        all_images.append(img / 255.0)

        # Assign label based on subdirectory
        if label == 'real':
            all_labels.append(0)
        elif label == 'fake':
            all_labels.append(1)


In [ ]:
all_images = np.array(all_images)
all_labels = np.array(all_labels)
all_images.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

In [ ]:
from huggingface_hub import from_pretrained_keras

reloaded_model = from_pretrained_keras('Emmawang/mobilenet_v2_fake_image_detection')

In [ ]:
reloaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
history1 = reloaded_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)

reloaded_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
history2 = reloaded_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# Assume 'model' is your trained Keras model
reloaded_model.save('/content/myModels/myModel.h5')


In [ ]:
from PIL import Image
import numpy as np

# Load and preprocess the image
image_path = '/content/aigen3.jpg'  # Replace with the path to your image
img = Image.open(image_path).resize([128, 128])
img_array = np.array(img).astype(np.float32) / 255.0


In [ ]:
from tensorflow.keras.models import load_model

reloaded_model = load_model('/content/myModels/myModel.h5')


img_array = np.expand_dims(img_array, axis=0)
prediction = reloaded_model.predict(img_array)


# New Section

In [ ]:
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np

# Load the pre-trained model
model = load_model('myModel.h5')

# Load and preprocess the image
img_path = '/content/human.webp'
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Example: Make predictions on the preprocessed image
predictions = model.predict(img_array)

# Interpret the result
binary_prediction = (predictions > 0.5).astype(int)
print("Predicted class:", binary_prediction[0])

